In [1]:
import pandas as pd
df = pd.read_csv("question_reproduce.csv")
df.head()

,Unnamed: 0,Question,openai/gpt-3.5-turbo,openai/gpt-4,meta-llama/llama-2-70b-chat,mistralai/mistral-7b-instruct
0,0,The StackADT's pop operation:,A) removes the bottom item from the stack\nB) ...,A) Returns the element at the top of the stack...,Which of the following statements best describ...,1. Retrieves the top element from the stack\n2...
1,1,You see the expression n = 100000 in some code...,A) char\nB) boolean\nC) float\nD) double\n\nCo...,A) int\nB) long\nC) float\nD) char (X),A) int\nB) long\nC) float\nD) double\n\n(X) B)...,A) An integer (X)\nB) A string\nC) A boolean\n...
2,2,public BallPanel extends javax.swing.JPanel {\...,A) System.out.println(_balls[0].getColor()); \...,A) this._balls[5].getColor()\nB) _balls[25].ge...,A) `_balls[0].getColor()` (X)\nB) `getBall(0)....,a) `_balls[i].setColor(_color);`\nb) `Ball[] a...
3,3,Suppose s is an instance of a stack that can s...,A. Finland (X)\n\nB. is\n\nC. my\n\nD. home,"A) ""is""\nB) ""Finland""\nC) ""home""\nD) ""my"" (X)","A) ""Finland""\nB) ""is""\nC) ""my""\nD) ""home""\n\n(...",1. (X) The value of z after executing these st...
4,4,Suppose you are trying to choose between an ar...,A) Queue\nB) Stack\nC) Heap\nD) Linked list (X),A) Stack\nB) Queue\nC) Array\nD) X) Linked List,A) Array\nB) Linked List\nC) Vector\nD) Stack\...,1. Arrays\n2. Linked List\n3. Stack\n4. Queue\...


In [2]:
import requests
import json
api_key = "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"
#"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"


def ask_question_open_router(question,model):
    response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {api_key}"
  },
  data=json.dumps({
    "model": model, 
    "messages": [
      {"role": "user", "content": question }
    ]
  })
)
    response_data = response.json()
    messages = [choice['message']['content'] for choice in response_data['choices']]
    return messages

In [3]:
Prompt = "Based on the following multiple-choice question, generate three plausible distractors and one correct answer. Mark the correct answer with an (X), and no explanation"

model = "anthropic/claude-3-opus"
answer = []
for i in range(50):
    question = Prompt + df['Question'][i]
    messages = ask_question_open_router(question,model)
    answer.append(messages[0])
df[model] = answer

KeyError: 'choices'

In [ ]:
Prompt = "Is the question an unambiguous exam question that has only one answer?, answer with YES or NO"
for i in range(50):
    question = Prompt + df['Question'][i]
    messages = ask_question_open_router(question,"anthropic/claude-3-opus")
    if "YES" in messages[0].upper():
        df['claude-3_unambiguous'][i] = "Y"
    else:
        df['claude-3_unambiguous'][i] = "N"

In [ ]:
prompt = "How would you rephrase the question to be unambiguous and suitable for a Multiple choice question on an exam? "
revised_question_claude3 = []
for i in range(len(df)):
    if df['claude-3_unambiguous'][i] == 'N':
        question = prompt+df['Question'][i]
        messages = ask_question_open_router(question,"anthropic/claude-3-opus")
        revised_question_claude3.append(messages[0])
    else:
        revised_question_claude3.append("None")
df['claude-3_revised_question'] = revised_question_claude3

In [ ]:
Prompt = "Based on the following multiple-choice question, generate three plausible distractors and one correct answer. Mark the correct answer with an (X), and no explanation"

answer = []
for i in range(50):
    question = Prompt + df['claude-3_revised_question'][i]
    messages = ask_question_open_router(question,model)
    answer.append(messages[0])
df["claude-3_revised_question_answer"] = answer